#### Ejercicio 2

In [1]:
import org.apache.spark.sql.DataFrame;
import org.apache.spark.util.SizeEstimator;
import org.apache.spark.sql.functions.desc;

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.33:4041
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1617079210812)
SparkSession available as 'spark'


import org.apache.spark.sql.DataFrame
import org.apache.spark.util.SizeEstimator
import org.apache.spark.sql.functions.desc


In [2]:
val DataScheme: DataFrame = spark.read.parquet("processed");

DataScheme: org.apache.spark.sql.DataFrame = [campaign_start_date: date, personal_type: string ... 4 more fields]


In [4]:
class EvaluateDataFrame (val dataScheme: DataFrame,
                         val prmSparkStatsLevel: Int,
                         val sparkPartitionId: String) {
    val partitions = dataScheme.rdd.getNumPartitions
    require(prmSparkStatsLevel > 0, """Spark level must be
    greater than 0""")
}

defined class EvaluateDataFrame


In [5]:
class DataframeSize(val df: DataFrame) {
    def toBytes(): BigInt = {
        df.queryExecution.optimizedPlan.stats.sizeInBytes
    }
    def toMb(size: Float): Float = {
        size.toFloat / 1024.toFloat / 1024.toFloat
    }
    def toEstimatorMb(): Float = {
        toMb(SizeEstimator.estimate(df))
    }
}

defined class DataframeSize


In [6]:
trait PrmLevel1 {
    def dfStats(df: EvaluateDataFrame) = {
        val dfSize = new DataframeSize(df.dataScheme: DataFrame)
        
        val sizeBytes: BigInt = dfSize.toBytes()
        val sizeMb: Float = dfSize.toMb(sizeBytes.toFloat)
        val sizeEstimatorMb: Float = dfSize.toEstimatorMb()
        
        println("*** STATS *** Df Stats PlanSizeMB:"+sizeMb+
                ",  EstimatorSizeMB:"+sizeEstimatorMb+
                ", PartitionNum: "+df.partitions+
                ", PartitionSizeMB:"+sizeMb / df.partitions+
                ", PartitionSizeMB:"+sizeEstimatorMb / df.partitions)
    }
}

defined trait PrmLevel1


In [7]:
trait PrmLevel2 extends PrmLevel1 {
    override def dfStats(df: EvaluateDataFrame) = {
        super.dfStats(df);
        println(s"""*** STATS *** Df Partition record 
               count ${df.dataScheme.count()}""");
    }
}

defined trait PrmLevel2


In [8]:
trait PrmLevel3 extends PrmLevel1 {
    override def dfStats(df: EvaluateDataFrame) = {
        super.dfStats(df);
        println(s"*** STATS *** Df Partition Info");
        df.dataScheme
          .groupBy(df.sparkPartitionId)
          .count()
          .orderBy(desc("count"))
          .show(if (df.partitions>0) df.partitions else 200);
    }
}

defined trait PrmLevel3


In [9]:
trait PrmLevel8 extends PrmLevel1 {
    override def dfStats(df: EvaluateDataFrame) = {
        super.dfStats(df);
        println(s"*** STATS *** Show data:")
        df.dataScheme.show() 
    }
}

defined trait PrmLevel8


In [10]:
class PrmSparkStatsLevel1 (val df: EvaluateDataFrame) extends PrmLevel1 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel1


In [11]:
class PrmSparkStatsLevel2 (val df: EvaluateDataFrame) extends PrmLevel2 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel2


In [12]:
class PrmSparkStatsLevel3 (val df: EvaluateDataFrame) extends PrmLevel3 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel3


In [13]:
class PrmSparkStatsLevel4 (val df: EvaluateDataFrame) extends PrmLevel2 with PrmLevel3 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel4


In [14]:
class PrmSparkStatsLevel8 (val df: EvaluateDataFrame) extends PrmLevel8 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel8


In [15]:
val df1: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 1, "cutoff_date")
val df2: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 2, "cutoff_date")
val df3: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 3, "cutoff_date")
val df4: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 4, "cutoff_date")
val df8: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 8, "cutoff_date")

df1: EvaluateDataFrame = EvaluateDataFrame@49275ea2
df2: EvaluateDataFrame = EvaluateDataFrame@42edf4f7
df3: EvaluateDataFrame = EvaluateDataFrame@3e9a1ed5
df4: EvaluateDataFrame = EvaluateDataFrame@7580424f
df8: EvaluateDataFrame = EvaluateDataFrame@6a1c013


In [16]:
val prm1 = new PrmSparkStatsLevel1(df1)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:5.2294617, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:1.3073654


prm1: PrmSparkStatsLevel1 = PrmSparkStatsLevel1@532876a7


In [17]:
val prm2 = new PrmSparkStatsLevel2(df2)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:5.233917, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:1.3084793
*** STATS *** Df Partition record 
               count 1282382


prm2: PrmSparkStatsLevel2 = PrmSparkStatsLevel2@a621d5b


In [18]:
val prm3 = new PrmSparkStatsLevel3(df3)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:5.502426, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:1.3756065
*** STATS *** Df Partition Info
+-----------+-------+
|cutoff_date|  count|
+-----------+-------+
| 2020-01-03|1282382|
+-----------+-------+



prm3: PrmSparkStatsLevel3 = PrmSparkStatsLevel3@5da07f36


In [19]:
val prm4 = new PrmSparkStatsLevel4(df4)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:14.470245, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:3.6175613
*** STATS *** Df Partition record 
               count 1282382
*** STATS *** Df Partition Info
+-----------+-------+
|cutoff_date|  count|
+-----------+-------+
| 2020-01-03|1282382|
+-----------+-------+



prm4: PrmSparkStatsLevel4 = PrmSparkStatsLevel4@2aca30d3


In [20]:
val prm8 = new PrmSparkStatsLevel8(df8)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:15.233841, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:3.8084602
*** STATS *** Show data:
+-------------------+-------------+-----------+------------------+--------------------+-----------+
|campaign_start_date|personal_type|personal_id|      segment_desc|   audtiminsert_date|cutoff_date|
+-------------------+-------------+-----------+------------------+--------------------+-----------+
|         2019-09-30|            L|   00081633|  08. Bajo ingreso|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   00950462|  08. Bajo ingreso|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   02446085|              None|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   06674137|              None|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   06694902|              None|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L

prm8: PrmSparkStatsLevel8 = PrmSparkStatsLevel8@3244b440
